In [12]:
import os
import zipfile

import pandas as pd

In [13]:
from tkgqa_generator.tkgqa_generator.constants import DATA_ICEWS_EVENTS_DATA_DIR

In [14]:
DATA_ICEWS_EVENTS_DATA_DIR

PosixPath('/Users/pascal/PhD/TemporalLogicKGQA/tkgqa_generator/data/ICEWS/ICEWS Coded Event Data')

In [17]:
# loop the folder, and unzip all of the files ending with .zip
for file in os.listdir(DATA_ICEWS_EVENTS_DATA_DIR):
    if file.endswith(".zip"):
        # unzip the file
        zip_path = DATA_ICEWS_EVENTS_DATA_DIR / file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DATA_ICEWS_EVENTS_DATA_DIR)

In [8]:
# pandas read from zip
df = pd.read_csv(DATA_ICEWS_EVENTS_DATA_DIR / "20230108-icews-events.tab", sep='\t')

In [10]:
df.columns

Index(['Event ID', 'Event Date', 'Source Name', 'Source Sectors',
       'Source Country', 'Event Text', 'CAMEO Code', 'Intensity',
       'Target Name', 'Target Sectors', 'Target Country', 'Story ID',
       'Sentence Number', 'Publisher', 'City', 'District', 'Province',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [11]:
df

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,37543936,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Deny responsibility,16,-5.0,United States,NaN,United States,56635750,4,The Oil Daily,NaN,NaN,Washington D.C.,United States,38.9171,-77.00030
1,37543653,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Praise or endorse,51,3.4,Russia,NaN,Russian Federation,56635751,2,The Oil Daily,Riyadh,NaN,Mintaqat ar Riyad,Saudi Arabia,24.6877,46.72190
2,37542178,2022-10-14,Israel,NaN,Israel,Provide aid,70,7.0,United States,NaN,United States,56633048,5,The Jerusalem Post,NaN,NaN,NaN,United States,38.8951,-77.03640
3,37542179,2022-10-14,United States,NaN,United States,Provide economic aid,71,7.4,Israel,NaN,Israel,56633048,6,The Jerusalem Post,NaN,NaN,NaN,Israel,31.7790,35.22530
4,37542131,2022-10-14,Ukraine,NaN,Ukraine,"Arrest, detain, or charge with legal action",173,-5.0,Volodymyr Zelensky,"Elite,Executive,Executive Office,Government",Ukraine,56633051,2,The Jerusalem Post,NaN,NaN,NaN,Ukraine,50.4547,30.52380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15609,37552500,2023-01-09,Court Judge (Brazil),"Government,Judicial",Brazil,Consult,40,1.0,Jair Bolsonaro,"Elite,Executive,Executive Office,Government",Brazil,56654472,5,ANSA - Spanish Service (BASP),Araraquara,Araraquara,Estado de Sao Paulo,Brazil,-21.7944,-48.17560
15610,37552547,2023-01-09,Ireland,NaN,Ireland,Praise or endorse,51,3.4,Brazil,NaN,Brazil,56654475,4,Unknown,Dublin,Dublin City,Leinster,Ireland,53.3331,-6.24889
15611,37552528,2023-01-09,Activist (Brazil),"Human Rights IGOs,Human Rights NGOs,Nongovernm...",Brazil,Demonstrate or rally,141,-6.5,Unspecified Actor,Unspecified,NaN,56654476,4,Agence France-Presse,NaN,NaN,NaN,Spain,40.4165,-3.70256
15612,37552602,2023-01-09,Democratic Party of Equatorial Guinea,"(National) Major Party,Parties",Equatorial Guinea,Make a visit,42,1.9,France,NaN,France,56654487,5,All Africa,Paris,Paris,Region Ile-de-France,France,48.8534,2.34880
